In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.activations import softplus
import joblib

df = pd.read_csv("/content/drive/MyDrive/Metro_Interstate_Traffic_Volume.csv")
df['date_time'] = pd.to_datetime(df['date_time'])
df.set_index('date_time', inplace=True)
df.fillna(0, inplace=True)

le_holiday = LabelEncoder()
df['holiday'] = df['holiday'].astype(str) # Convert to string before encoding
df['holiday'] = le_holiday.fit_transform(df['holiday'])

le_weather = LabelEncoder()
df['weather_main'] = le_weather.fit_transform(df['weather_main'])
df['weather_description'] = le_weather.fit_transform(df['weather_description'])

Runtime Features
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
df['is_rush_hour'] = df['hour'].isin([7,8,16,17]).astype(int)

Cyclic encoding
df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

Lag features
df['traffic_lag1'] = df['traffic_volume'].shift(1).fillna(method='bfill')
df['traffic_lag2'] = df['traffic_volume'].shift(2).fillna(method='bfill')

Rolling averages
df['traffic_rolling3'] = df['traffic_volume'].rolling(3).mean().fillna(method='bfill')
df['rain_3h'] = df['rain_1h'].rolling(3).mean().fillna(0)
df['snow_3h'] = df['snow_1h'].rolling(3).mean().fillna(0)
df['temp_3h'] = df['temp'].rolling(3).mean().fillna(df['temp'])

Features and target
features = [
    'holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all',
    'weather_main', 'weather_description',
    'hour', 'day_of_week', 'month', 'is_weekend', 'is_rush_hour',
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
    'traffic_lag1', 'traffic_lag2',
    'traffic_rolling3', 'rain_3h', 'snow_3h', 'temp_3h'
]
target = ['traffic_volume']

X = df[features].values
y = df[target].values

Normalization
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

Sequences
SEQ_LENGTH = 10
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)-seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X_scaled, y_scaled, SEQ_LENGTH)

Split train/test
train_size = int(len(X_seq)*0.8)
X_train, X_test = X_seq[:train_size], X_seq[train_size:]
y_train, y_test = y_seq[:train_size], y_seq[train_size:]

LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(SEQ_LENGTH, X_seq.shape[2])))
model.add(Dense(1, activation=softplus))
model.compile(optimizer='adam', loss='mse')

Train
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1)
model.save("models/traffic_lstm_model.h5")

Save scalers
joblib.dump(scaler_X, "models/scaler_X.pkl")
joblib.dump(scaler_y, "models/scaler_y.pkl")

Predict
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_pred = np.maximum(0, y_pred)

/tmp/ipython-input-515487597.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['traffic_lag1'] = df['traffic_volume'].shift(1).fillna(method='bfill')
/tmp/ipython-input-515487597.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['traffic_lag2'] = df['traffic_volume'].shift(2).fillna(method='bfill')
/tmp/ipython-input-515487597.py:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['traffic_rolling3'] = df['traffic_volume'].rolling(3).mean().fillna(method='bfill')
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model inst

Epoch 1/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0260 - val_loss: 0.0077
Epoch 2/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0089 - val_loss: 0.0074
Epoch 3/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0069 - val_loss: 0.0037
Epoch 4/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0062 - val_loss: 0.0038
Epoch 5/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - loss: 0.0055 - val_loss: 0.0035
Epoch 6/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - loss: 0.0055 - val_loss: 0.0033
Epoch 7/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - loss: 0.0050 - val_loss: 0.0032
Epoch 8/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.0049 - val_loss: 0.0035
Epoch 9/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0044 - val_loss: 0.0034
Epoch 10/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.0044 - val_loss: 0.0032
Epoch 11/30
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0045 - val_loss: 0.0029
Epoch 12/30
1085/108

302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
